<a href="https://colab.research.google.com/github/maheshh-v/groq-api-assignment/blob/main/groq_api_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Groq API Assignment  
By Mahesh Vyas  

This notebook shows two tasks using the Groq API without any frameworks:  
1. Manage conversation history (truncation + summarization).  
2. Extract user details into JSON using function calling.  

my goal is to keep it simple, clear, and working.

----------------
### **How to Run This Notebook**
To execute the code, please add your Groq API key to the Colab Secrets manager:
1.  Click the **Key   icon** on the left sidebar.
2.  create a new secret with the name `GROQ_API_KEY`.
3.  Paste your API key into the value

In [28]:
# 1- Setup and Dependencies ---
!pip install groq openai -q

import os
import json
from groq import Groq
from google.colab import userdata

# 2- API Client Initialization --
# Get the API key from Colab's secret manager
try:
    GROQ_API_KEY = userdata.get('GROQ_API_KEY')
    print("Successfully retrieved API key.")
except Exception as e:
    print("Could not retrieve API key. Please follow the instructions above to add it to Colab Secrets.")
    GROQ_API_KEY = None

# Initializing the Groq client
if GROQ_API_KEY:
    client = Groq(api_key=GROQ_API_KEY)
    print("Groq client initialized.")

Successfully retrieved API key.
Groq client initialized.


## Task 1: Managing Conversation History

Here I’m trying to make a chatbot remember chats without storing everything forever.  
Two tricks:  
1. **Truncation** → only keep the last N turns.  
2. **Summarization** → after a few turns, shrink old chats into a short summary.  


### Part 1(a): Truncation by Number of Turns

This is the quick way: just keep the last N turns.  
Fast and simple, but older context gets cut.  
Below I show it with N=2.  


In [25]:
# --- Function to truncate history by a fixed number of turns ---
def truncate_history_by_turns(history, max_turns=2):
    """
    Keeps the system message and the most recent `max_turns` of the conversation.
    A turn consists of one user message and one assistant message.
    """
    if len(history) - 1 <= max_turns * 2:
        return history

    system_message = history[0]
    conversation = history[1:]
    truncated_conversation = conversation[-(max_turns * 2):]
    return [system_message] + truncated_conversation

#   Loop for Truncation (Corrected) ---
print("--- Starting Truncation Demo (Max 2 Turns) ---")
print("--- Have a 5-6 turn conversation to see the effect. Type 'exit' to stop. ---")

truncation_history = [{'role': 'system', 'content': 'You are a helpful assistant.'}]

turn_counter = 1
while True:
    user_input = input(f"You (Turn {turn_counter}): ")
    if user_input.lower() in ["exit", "quit"]:
        break

    truncation_history.append({'role': 'user', 'content': user_input})

    #  FIX ---
    # We now overwrite the main history list with the truncated version.
    truncation_history = truncate_history_by_turns(truncation_history, max_turns=2)

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=truncation_history
    )
    assistant_response = response.choices[0].message.content

    truncation_history.append({'role': 'assistant', 'content': assistant_response})

    print(f"Assistant: {assistant_response}")
    print(f"--- History Length: {len(truncation_history)} messages ---\n")
    turn_counter += 1

--- Starting Truncation Demo (Max 2 Turns) ---
--- Have a 5-6 turn conversation to see the effect. Type 'exit' to stop. ---
You (Turn 1): is switzerland good place to live?
Assistant: Switzerland is often considered one of the best places to live in the world, and here's why:

1. **High standard of living**: Switzerland has a high standard of living, with excellent infrastructure, education, and healthcare systems.
2. **Safety and stability**: Switzerland is considered one of the safest countries in the world, with low crime rates and a strong economy.
3. **Natural beauty**: Switzerland is famous for its stunning mountains, lakes, and forests, making it a paradise for nature lovers and outdoor enthusiasts.
4. **Cultural diversity**: Switzerland has a rich cultural heritage, with four official languages (German, French, Italian, and Romansh) and a vibrant arts scene.
5. **Financial stability**: Switzerland is known for its financial stability and secure banking system, making it an attr

### Part 1(b): Periodic Summarization

Instead of throwing old chats away, I ask the model to summarize every 3 turns.  
That way it still remembers the main points, but the history stays short.  


In [41]:
# make the model shrink old chats into a short summary
# useful when convo gets long (I trigger it every 3 turns here)

def summarize_conversation(history):
    """
    Ask the LLM to shrink the full conversation into a short summary.
    Keeps context but saves memory.
    """
    print("\n--- Summarizing chat so far... ---")

    conversation_text = "\n".join([f"{msg['role']}: {msg['content']}" for msg in history])

    summarization_prompt = f"""
    Summarize this chat in 1 short paragraph.
    Keep only the important facts and user goals.

    Chat:
    {conversation_text}
    """

    summary_completion = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": summarization_prompt}],
    )
    summary_text = summary_completion.choices[0].message.content.strip()

    # Replace history with a short system note
    new_history = [
        history[0],
        {"role": "system", "content": f"Summary so far: {summary_text}"}
    ]
    print("--- Done summarizing ---\n")
    return new_history


# Chat loop with auto-summarization -
print("--- Chatbot (summarizes every 3 turns) ---")
print("Type messages, watch it summarize on turns 3, 6, 9... Type 'exit' to stop.")

conversation_history = [{'role': 'system', 'content': 'You are a helpful assistant.'}]
run_counter = 0
SUMMARIZE_EVERY_N = 3

while True:
    user_input = input(f"You (Turn {run_counter + 1}): ")
    if user_input.lower() in ["exit", "quit"]:
        break

    run_counter += 1
    conversation_history.append({'role': 'user', 'content': user_input})

    # at fixed intervals
    if run_counter % SUMMARIZE_EVERY_N == 0:
        conversation_history = summarize_conversation(conversation_history)
        conversation_history.append({'role': 'user', 'content': user_input})

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=conversation_history,
    )
    assistant_response = response.choices[0].message.content

    conversation_history.append({'role': 'assistant', 'content': assistant_response})

    print(f"Assistant: {assistant_response}")
    print(f"(History length: {len(conversation_history)})\n")


--- Chatbot (summarizes every 3 turns) ---
Type messages, watch it summarize on turns 3, 6, 9... Type 'exit' to stop.
You (Turn 1): who is the richest person in india?
Assistant: As of my knowledge cutoff in 2023, the richest person in India is Gautam Adani. He is an Indian industrialist and entrepreneur who is the founder and chairman of the Adani Group, a multinational conglomerate with diverse business interests in various sectors such as energy, transportation, real estate, and agriculture.

According to Forbes, Gautam Adani's net worth is approximately $143 billion as of March 2023.
(History length: 3)

You (Turn 2): how much he earns per second??
Assistant: To estimate how much Gautam Adani earns per second, we'll need to make a few assumptions and calculations. 

First, let's assume an annual salary or income of $143 billion, which is his net worth as of 2023.

Next, let's assume he earns this amount evenly throughout the year, which is not possible in reality, but we'll use thi

## Task 2: JSON Info Extraction

Now I test pulling clean data (name, email, phone, location, age) out of messy chat text.  
I define a JSON schema, feed in some sample chats, and check if the output matches.  


In [43]:
import json

# This is a simple schema to help us pick out user details from text
USER_SCHEMA = {
    "type": "function",
    "function": {
        "name": "extract_user_info",
        "description": "pulls out user info like name, email, phone, location, and age from a text snippet",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {"type": "string", "description": "Full name of the person."},
                "email": {"type": "string", "description": "Email address."},
                "phone": {"type": "string", "description": "Phone number."},
                "location": {"type": "string", "description": "City and state."},
                "age": {"type": "string", "description": "Age in years."}
            },
            "required": ["name", "email"]
        }
    }
}

def get_user_info(text):
    """
    Extracts user details from a text snippet using the Groq API.
    Returns a dictionary with the extracted info.
    """
    try:
        prompt = """You are a smart assistant.
        Read the text and extract the user's details using the 'extract_user_info' tool
        If something is missing, leave it as an empty string, No extra commentary needed"""

        response = client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": text}
            ],
            tools=[{"type": "function", "function": USER_SCHEMA["function"]}],
            tool_choice="auto"  # Let the model pick the tool automatically
        )

        # Grab what the model returned via the tool
        tool_calls = response.choices[0].message.tool_calls
        if not tool_calls:
            return None

        info = json.loads(tool_calls[0].function.arguments)

        # Make age an integer if possible
        if info.get('age'):
            try:
                info['age'] = int(info['age'])
            except (ValueError, TypeError):
                pass  # Keep it as string if conversion fails

        return info

    except Exception as e:
        print(f" Something went wrong while extracting info: {e}")
        return None


if __name__ == "__main__":
    test_texts = [
        "Hi, Im Mahesh Vyas from Udaipur. my email is maheshvya.724@gmail.com, phone 9672435098, and I am 22",
        "Hello, Im Anjali Sharma. email anjali.SS@example.com, age 30",
        "My name is Rohan, based in Mumbai Contact: rohan.k@work.net, phone 12 7890."
    ]

    print("--- Testing User Info Extraction ---")
    for idx, text in enumerate(test_texts):
        print(f"\nProcessing sample #{idx + 1}: {text}")
        result = get_user_info(text)
        if result:
            print("Info extracted:")
            print(json.dumps(result, indent=4))
        else:
            print(" failed.")


--- Testing User Info Extraction ---

Processing sample #1: Hi, Im Mahesh Vyas from Udaipur. my email is maheshvya.724@gmail.com, phone 9672435098, and I am 22
Info extracted:
{
    "age": 22,
    "email": "maheshvya.724@gmail.com",
    "location": "Udaipur",
    "name": "Mahesh Vyas",
    "phone": "9672435098"
}

Processing sample #2: Hello, Im Anjali Sharma. email anjali.SS@example.com, age 30
Info extracted:
{
    "age": 30,
    "email": "anjali.SS@example.com",
    "location": "",
    "name": "Anjali Sharma",
    "phone": ""
}

Processing sample #3: My name is Rohan, based in Mumbai Contact: rohan.k@work.net, phone 12 7890.
Info extracted:
{
    "age": "",
    "email": "rohan.k@work.net",
    "location": "Mumbai",
    "name": "Rohan",
    "phone": "12 7890"
}


## Conclusion

Both tasks worked:  
- The chatbot can either cut history or summarize it.  
- The model can return user details as clean JSON.  

I kept the code simple so it’s easy to follow and test.  
